In [1]:
from pynq import Overlay, GPIO, Register, allocate, MMIO
import os
import numpy as np
import struct


In [2]:
overlay = Overlay("nn.bit")
fcc1=overlay.forward_fcc_0
fcc1.register_map

bck1=overlay.backward_fcc_0
bck1.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=0),
  w = Register(w=0),
  y = Register(y=0),
  b = Register(b=0),
  xdimension = Register(xdimension=0),
  ydimension = Register(ydimension=0)
}

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=0),
  w = Register(w=0),
  b = Register(b=0),
  dx = Register(dx=0),
  dy = Register(dy=0),
  xdimension = Register(xdimension=0),
  ydimension = Register(ydimension=0),
  lr = Register(lr=0)
}

In [ ]:
class FullyConnectedLayer():
    def __init__(self,xdim,ydim,base_addr):
        self.xdim=xdim
        self.ydim=ydim

        self.base_addr=base_addr

        self.BASE_ADDRESS_X=base_addr
        self.BASE_ADDRESS_DX=self.BASE_ADDRESS_X+xdim*4+4

        self.BASE_ADDRESS_W=self.BASE_ADDRESS_DX+xdim*4+4

        self.BASE_ADDRESS_B=self.BASE_ADDRESS_W+xdim*ydim*4+4

        self.BASE_ADDRESS_Y=self.BASE_ADDRESS_B+ydim*4+4
        self.BASE_ADDRESS_DY=self.BASE_ADDRESS_Y+ydim*4+4  

        self.mmio_x= MMIO(self.BASE_ADDRESS_X,self.xdim*4)
        self.mmio_dx=MMIO(self.BASE_ADDRESS_DX,self.xdim*4)

        self.mmio_w= MMIO(self.BASE_ADDRESS_W,self.xdim*self.ydim*4)

        self.mmio_y= MMIO(self.BASE_ADDRESS_Y,self.ydim*4)
        self.mmio_dy= MMIO(self.BASE_ADDRESS_DY,self.ydim*4)

        self.mmio_b= MMIO(self.BASE_ADDRESS_B,self.ydim*4)

        self.config_dic={'base':base_addr, 'x':self.BASE_ADDRESS_X,'w':self.BASE_ADDRESS_W,'y':self.BASE_ADDRESS_Y,'b':self.BASE_ADDRESS_B, 'dx':self.BASE_ADDRESS_DX, 'dy':self.BASE_ADDRESS_DY,'xdim':self.xdim,'ydim':self.ydim}

    def get_config_dic(self):
        
        return self.config_dic

    def set_config_from_dic(self,dic):

        self.base_addr=dic['base']

        self.BASE_ADDRESS_X=dic['x']
        self.BASE_ADDRESS_DX=dic['dx']

        self.BASE_ADDRESS_W=dic['w']

        self.BASE_ADDRESS_B=dic['b']

        self.BASE_ADDRESS_Y=dic['y']
        self.BASE_ADDRESS_DY=dic['dy']


        self.mmio_x= MMIO(self.BASE_ADDRESS_X,self.xdim*4)
        self.mmio_dx=MMIO(self.BASE_ADDRESS_DX,self.xdim*4)

        self.mmio_w= MMIO(self.BASE_ADDRESS_W,self.xdim*self.ydim*4)

        self.mmio_y= MMIO(self.BASE_ADDRESS_Y,self.ydim*4)
        self.mmio_dy= MMIO(self.BASE_ADDRESS_DY,self.ydim*4)

        self.mmio_b= MMIO(self.BASE_ADDRESS_B,self.ydim*4)

        self.xdim=dic['xdim']
        self.ydim=dic['ydim']

        self.config_dic=dic


    def initHardware(self,fwip,bckip,lr):

        self.fwip=fwip
        self.fwip.register_map.x=self.BASE_ADDRESS_X
        self.fwip.register_map.w=self.BASE_ADDRESS_W
        self.fwip.register_map.y=self.BASE_ADDRESS_Y
        self.fwip.register_map.b=self.BASE_ADDRESS_B
        self.fwip.register_map.xdimension=self.xdim
        self.fwip.register_map.ydimension=self.ydim

        self.bckip= bckip
        self.bckip.register_map.x=self.BASE_ADDRESS_X
        self.bckip.register_map.w=self.BASE_ADDRESS_W
        self.bckip.register_map.b=self.BASE_ADDRESS_B
        self.bckip.register_map.xdimension=self.xdim
        self.bckip.register_map.ydimension=self.ydim
        self.bckip.register_map.dx=self.BASE_ADDRESS_DX
        self.bckip.register_map.dy=self.BASE_ADDRESS_DY
        self.bckip.register_map.lr=lr

    def reset_weights(self):

        for i in range(self.xdim*self.ydim):
            self.mmio_w.write(i*4,np.random.random())
        
        for i in range(self.ydim):
            self.mmio_b.write(i*4,np.random.random())

    def reset_input(self):
        for i in range(self.xdim):
            self.mmio_x.write(i*4,0)
            self.mmio_dx.write(i*4,0)

    def reset_output(self):
        for i in range(self.ydim):
            self.mmio_y.write(i*4,0)
            self.mmio_dy.write(i*4,0)

    def set_input(self,x):
        for i in range(self.xdim):
            self.mmio_x.write(i*4,x[i])
    
    def get_output(self):
        y=[]
        for i in range(self.ydim):
            y.append(self.mmio_y.read(i*4))
        return y

    def set_dy(self,dy):
        for i in range(self.ydim):
            self.mmio_dy.write(i*4,dy[i])

    def fwprop(self):

        self.fwip.write(0x00, 1)
        fpga_state = self.fwip.read(0x00)

        max_try = 1000000
        while fpga_state != 6 and fpga_state != 4:
            fpga_state = self.fwip.read(0x00)
            max_try = max_try -1
            if max_try == 0:
                print("ERROR: Can't go ahead")
                self.fwip.write(0x00, 4)
                break

        self.fwip.write(0x00, 4)

    def bckprop(self):
        self.bckip.write(0x00, 1)
        fpga_state = self.bckip.read(0x00)

        max_try = 1000000
        while fpga_state != 6 and fpga_state != 4:
            fpga_state = self.bckip.read(0x00)
            max_try = max_try -1
            if max_try == 0:
                print("ERROR: Can't go ahead")
                self.bckip.write(0x00, 4)
                break

        self.bckip.write(0x00, 4)
        


In [ ]:
class Neural_Net():

    def __init__(self,mem_base_addr):

        self.mem_base_addr=mem_base_addr
        self.layers=[]
        self.last_layer_out_address=mem_base_addr
        self.layer_base_addresses=[]
        self.layer_configs=[]

    def add(self,input_shape,output_shape):
        layer=FullyConnectedLayer(input_shape,output_shape,self.last_layer_out_address)
        self.layers.append(layer)
        self.last_layer_out_address=layer.config_dic['y']
        layer.reset_weights()
        self.layer_configs.append(layer.config_dic)
       
    
    def get_layer_output(self,i):

        return self.layers[i].get_output()
    
    def runfwprop(self,x):

        self.layers[0].set_input(x)

        for i in range(self.nlayers):
            self.layer[i].initHardware(fcc1,bck1)
            self.layers[i].fwprop()

    def runbackprop(self,ylabel,learning_rate):

        pred=self.layers[self.nlayers-1].get_output()
        grad=self.calculate_gradient(pred,ylabel)

        self.layers[self.nlayers-1].set_dy(grad)
        
        for i in range(self.nlayers):
            self.layers[self.nlayers-i-1].initHardware(fcc1,bck1,learning_rate)
            self.layers[self.nlayers-i-1].bckprop()

    
    def get_weights(self):
        weights=[]
        for i in range(self.nlayers):
            weights.append(self.layers[i].get_weights())
        return weights

    def set_weights(self,weights):        
        for i in range(self.nlayers):
            self.layers[i].set_weights(weights[i])
    
    def get_output(self):           
        y=[]
        for i in range(self.nlayers):
            y.append(self.layers[i].get_output())
        return y
    
    def calculate_mse_loss(self,y,y_):
        return np.sum(np.square(y-y_))/len(y)

    def calculate_gradient(self,y,ytrue):
        return 2*(y-ytrue)/len(y)

    def train(self,x,y,epochs,learning_rate):
        for i in range(epochs):
            for j in range(len(x)):
                self.runfwprop(x[j])
                self.runbackprop(y[j],learning_rate)

    def predict(self,x):
        self.runfwprop(x)
        return self.get_output()[self.nlayers-1]        

In [ ]:

import numpy as np

x=np.range(1000)*0.01
y=np.sin(x)

model=Neural_Net(0x4001_0000)
model.add(1,8)
model.add(8,1)

model.train(x,y,20,0.01)